# Knapsack Problem

First, we created a dataset for a 0/1 Knapsack Problem using the "create_KSP" funcion. 
The goal is to select a subset of items to maximize the total utility while adhering to a constraint on the total weight (the capacity of the knapsack).
We defined the function with number_of_items, gave set a "max_utility" of 100, representing the maximum value that an item can have, as well as a maximum weight (max_weight) of 100. 
The function returns a tuple of two arrays, "utility" and "capacity".

The objective it's to evaluate the fitness of the solution, indicating how well the solution aligns the objectives of the problem. In this context, the aim is to optimize of the code to maximize the total value (utility) of the selected items while not exceeding the capacity (total weight) of the knapsack.

In [35]:
import numpy as np
np.random.seed(10)


def create_ksp(number_of_items, max_utility = 100, max_weight = 100):
    utility = np.random.randint(0, max_utility, number_of_items)
    capacity = np.random.randint(0, max_utility, number_of_items) #Ensure capacity is not zero
    
    return utility, capacity
    
utility, capacity = create_ksp(100)
utility #this utility generates an array with 100 items, each having randomly assigned utility and weight values. 
capacity 


array([ 3, 29, 16, 84, 82, 14, 51, 79, 17, 50, 53, 25, 48, 17, 32, 81, 80,
       41, 90, 12, 30, 81, 17, 16,  0, 31, 73, 64, 38, 22, 96, 66, 67, 62,
       95, 99, 27, 82, 62, 77, 48, 93, 75, 86, 37, 11, 21, 33, 95, 43, 88,
       96, 73, 40, 43, 90, 71,  8, 85, 72, 28, 30, 89, 25, 78, 81, 85, 62,
       13, 41, 33,  4, 87, 94, 28, 39, 91,  9,  7, 22, 32,  3,  9, 52, 76,
       68, 30, 70, 74, 30,  9,  2, 65, 13, 75, 52,  5, 93, 84, 48])

We now create a function that is responsible for generating an initial population of binary vectors. Each vector represents a potential solution for the problem at hand,
where each element is either 0 or 1, indicating whether an item is included or excluded from the knapsack.

The "objective_function" calculates the fitness of a given solution. The objective value is the total utility of the selected items, provided that the total weight is within the knapsack's capacity; otherwise it returns 0. 

In the "tournament_selection" we choose individuals from a population to act as parents for the genetic operations (crossover and mutation). It is termed a tournament in the genetic algorithm because it envolves small tournaments amongst a few individuals randomly chosen from the population. The individual with the highest fitness in the tournament is selected as a parent, in other words, the one with the highest fitness (total utility of items respecting the knapsacks weight) is chosen as a parent. 

In the crossover function, we simulate the process of exchanging genetic material between two parents (parent 1 and parent 2) to generate a new solution (child).
Withing the function, the "crossover_point" determines the position in the binary vector of the parents where the genetic material is exchanged to create a new individual. This introduces diversity into the population, ensuring that the offspring exhibits variation in its genetic makeup. This variation allows for different combinations of traits and enhances the algorithm's exploration of the solution space.

After the crossover we define the "mutation" function. The purpose of mutation is to ensure diversity in the population and prevent premature convergence to supoptimal solutions. The higher the "mutation_rate", the greater the randomness and exploration introduced into the population.


In [36]:
import numpy as np
np.random.seed(10)


def initialize_population(population_size, num_items):
    return np.random.randint(2, size=(population_size, num_items)) #each element is randomly chosen from the range 0 or 1.

def objective_function(utility, capacity, max_capacity, choice):
    total_weight = np.dot(capacity, choice) #calculates the total weight of the items selected in the solution 
    if total_weight <= max_capacity:
        return np.dot(utility, choice) 
    else:
        return 0 #checking whether the total weight is within the knapsack's 'max_capacity', returing the total utility of the selected items if it is, returns zero if not.

def tournament_selection(population, fitness, tournament_size):
    indices = np.random.choice(len(population), size=tournament_size, replace=False) #selects indices (individuals) from the population without replacement
    tournament_fitness = fitness[indices] #extracts the fitness values of the individuals participating in the tournament. It's an array containing the fitness values
    return population[indices[np.argmax(tournament_fitness)]] #retuns the binary vector (solution) of the individual with the highest fitness in the tournament. 
#this binary vector represents the solution selected as parent. 

def crossover(parent1, parent2):
    crossover_point = np.random.randint(len(parent1)) #randomly selects a crossover point within the length of the vector.
    child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:])) #creates a new binary vector "child" by combining the genetic material of the parents.
    return child

def mutate(solution, mutation_rate):
    mutation_indices = np.random.rand(len(solution)) < mutation_rate #this line creates a binary array. Generates a random value between 0 and 1 for each element in the array.
    solution[mutation_indices] = 1 - solution[mutation_indices] #Performs the mutation operation. It flips the bits at the mutation where "mutation_indices" is True. If the original value was 1, it becomes 0, and so fourth.
    return solution


After defining each function, we proceed to exploit with the Genetic Algorithm. With the parameters, we characterize the genetic algorithm and the problem. Next, we create the initial population that will undergo evolution over multiple generations.

The loop iterates through multiple generations, and in each generation, it evaluates the fitness of the population, selects pairs of parents using tournament selection, applies crossover to create children (child), mutates each child and forms a new population for the next generation. The algorithm aims to evolve a population of binary vectors that represents high-quality solutions to the Knapsack problem.

To conclude, the final part of the code extracts and evaluates the best solution found by the Genetic ALgorithm. It prints both the binary vector, representing the best solution, and the corresponding total value and weight of the items in the knapsack. 

In [37]:
np.random.seed(10)

# Parameters
num_items = 100
max_utility = 100
max_weight = 100
max_capacity = 2500
population_size = 50
num_generations = 1000
tournament_size = 5
crossover_rate = 0.8
mutation_rate = 0.01

# Initialization
population = initialize_population(population_size, num_items) #function to create an initial population of binary vectors, where each bit represents an included (1) or excluded (0) item in the knapsack

# Genetic Algorithm Loop
for generation in range(num_generations):
    fitness = np.array([objective_function(utility, capacity, max_capacity, individual) for individual in population]) #evaluates the fitness of each individual in the current population. The fitness array holds the fitness values.

    # Selection
    new_population = [] #new empty list
    for _ in range(population_size // 2): #iteration through half of the population size
        parent1 = tournament_selection(population, fitness, tournament_size)
        parent2 = tournament_selection(population, fitness, tournament_size) #selects two parents for crossovers. The tournament_size determines the number of individuals 

        #decides whether to apply crossover based on a randomly generated probability (crossover_rate). If crossover is chosen, two children are created by mixing the genetic material of the parents, otherwise, the children are direct copies of the parents.
        if np.random.rand() < crossover_rate:
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
        else:
            child1, child2 = parent1.copy(), parent2.copy()

        new_population.extend([child1, child2]) #adds the child objects to the new list.

    population = np.array(new_population) #updates the population to be the new population created through selection, crossover and mutation. 

# Find the best solution after the Genetic Algorithm
best_solution = population[np.argmax(fitness)] #This index corresponds to the position of the individual with the highest fitness in the array
best_value = objective_function(utility, capacity, max_capacity, best_solution) #calculates the total value of the items in the knapsack based on the binary vector of the best solution

print("Best Solution (after Genetic Algorithm):", best_solution)
print("Best Value (after Genetic Algorithm):", best_value)

#calculating the total weight of the items in the best solution
total_weight_of_best_solution = np.dot(capacity, best_solution)
print("Total Weight of the Best Solution:", total_weight_of_best_solution)


Best Solution (after Genetic Algorithm): [0 0 1 0 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1
 1 1 1 0 0 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 0]
Best Value (after Genetic Algorithm): 2907
Total Weight of the Best Solution: 2390


The Genetic Algorithm demonstrated its effectiveness in discovering near-optimal solutions through a combination of mutation and crossover operations. The one-point crossover introduced diversity, and the mutation operation added an essential element of randomness. Together, these operations enabled the Genetic Algorithm to adapt over generations and tuning toward the population with the highest fitness, whilst maintaining genetic diversity. 

---------------

Now, what happens if the added item is the one with the highest utility to weight ration in all iterations? Does this result in a better solution than the previous method?

The for loop in this function iterates through items sorted in descending order of utility-to-weight ratios. It adds items to the knapsack, maximizing total utility while staying within the capacity limit (max_capacity). The loop terminates when adding an item would exceed the capacity. The resulting knapsack solution, total utility, and total weight are then returned.

In [38]:
np.random.seed(10)

def ratio_knapsack(utility, capacity, max_capacity):
    num_items = len(utility)
    ratios = utility / capacity  # Calculate the utility-to-weight ratio for each item
    item_order = np.argsort(ratios)[::-1]  # Sort items in descending order of ratios

    knapsack = np.zeros(num_items, dtype=int)  # Initialize knapsack as an array of zeros
    total_weight = 0
    total_utility = 0

    for item in item_order:
        if total_weight + capacity[item] <= max_capacity:
            knapsack[item] = 1  # Add the item to the knapsack
            total_weight += capacity[item]
            total_utility += utility[item]

    return knapsack, total_utility, total_weight

#Example usage:
ratio_solution, ratio_value, ratio_weight = ratio_knapsack(utility, capacity, max_capacity)

print("Ratio Solution:", ratio_solution)
print("Ratio Value:", ratio_value)
print("Ratio Total Weight:", ratio_weight)

Ratio Solution: [1 0 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 0 1 0 0 0 1
 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0]
Ratio Value: 4031
Ratio Total Weight: 2482


C:\Users\maria\AppData\Local\Temp\ipykernel_720\3604083605.py:5: RuntimeWarning: divide by zero encountered in divide
  ratios = utility / capacity  # Calculate the utility-to-weight ratio for each item


---------------------

## Conclusion

The ratio-based algorithm, although achieving a higher utility value in this specific instance, doesn't ensure an optimal solution in all scenarios. In situations like having limited remaining capacity in the knapsack, this method may prioritize items with higher utility-to-weight ratios, potentially overlooking combinations that lead to more optimal solutions. For example, with 4 remaining capacity and three items with utility-weight pairs (Item1: 6-2.2; Item2: 4-2; Item3: 4-2), the ratio-based algorithm might choose the first item due to its higher ratio, even though selecting the second and third items would maximize utility (8 points) and fully utilize the knapsack's capacity. This showcases a limitation of the ratio-based approach, emphasizing the need to consider specific problem constraints and characteristics when selecting an optimization method.